In [4]:
import cv2  # OpenCV - biblioteka do przetwarzania obrazu
import dlib  # Biblioteka do detekcji twarzy i analizy punktów charakterystycznych
import numpy as np  # Obsługa operacji na macierzach
import imutils  # Pomocnicze operacje na obrazach (np. zmiana rozmiaru)
import os
import pandas as pd
import tkinter as tk
from tkinter import filedialog, simpledialog
import matplotlib.pyplot as plt  # Biblioteka do wizualizacji
from scipy.spatial import distance as dist  # Obliczanie odległości euklidesowej

def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    return (A + B) / (2.0 * C)

EYE_AR_THRESH = 0.25
EYE_AR_CONSEC_FRAMES = 20
blink_counter = 0
drowsy_alert = False
FRAME_WIDTH = 640
FRAME_HEIGHT = 480
frame_skip = 5
frames_folder = "frames"
os.makedirs(frames_folder, exist_ok=True)

def start_analysis(source):
    global blink_counter, drowsy_alert  
    blink_counter = 0  
    drowsy_alert = False  

    if source == "Kamera":
        cap = cv2.VideoCapture(0)
    elif source == "Wideo":  # Poprawione! elif przed else
        video_path = filedialog.askopenfilename(title="Wybierz plik wideo", filetypes=[("Pliki MOV", "*.MOV")])
        if video_path:  # Sprawdzenie, czy użytkownik wybrał plik
            cap = cv2.VideoCapture(video_path)
        else:
            print("Nie wybrano pliku.")
            return
    else:
        print("Niepoprawny wybór.")
        return
    
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor("C:\\Users\\Administrator\\projektzmeczenie\\shape_predictor_68_face_landmarks.dat")
    
    results = []
    frame_count = 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        if frame_count % frame_skip == 0:
            frame_path = os.path.join(frames_folder, f"frame_{frame_count}.jpg")
            cv2.imwrite(frame_path, frame)
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = detector(gray)
            EAR = 0

            for face in faces:
                landmarks = predictor(gray, face)
                left_eye = np.array([(landmarks.part(i).x, landmarks.part(i).y) for i in range(36, 42)])
                right_eye = np.array([(landmarks.part(i).x, landmarks.part(i).y) for i in range(42, 48)])
                left_EAR = eye_aspect_ratio(left_eye)
                right_EAR = eye_aspect_ratio(right_eye)
                EAR = (left_EAR + right_EAR) / 2.0
                
                if EAR < EYE_AR_THRESH:
                    blink_counter += 1
                else:
                    if blink_counter >= EYE_AR_CONSEC_FRAMES:
                        drowsy_alert = True
                    blink_counter = 0
            
            results.append([frame_count, frame_path, EAR, drowsy_alert])
            frame_resized = cv2.resize(frame, (FRAME_WIDTH, FRAME_HEIGHT))
            cv2.putText(frame_resized, f"Sleepy: {drowsy_alert}", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            cv2.imshow("Analiza Zmęczenia", frame_resized)
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        
        frame_count += 1
    
    cap.release()
    cv2.destroyAllWindows()
    
    df = pd.DataFrame(results, columns=["Frame", "Path", "EAR", "Drowsy"])
    df.to_csv("results.csv", index=False)
    print("Analiza zakończona. Wyniki zapisane do results.csv")

root = tk.Tk()
root.title("Wybór źródła wideo")
root.geometry("300x150")

tk.Label(root, text="Wybierz źródło wideo:").pack(pady=10)
source_var = tk.StringVar()
source_var.set("Kamera")

dropdown = tk.OptionMenu(root, source_var, "Kamera", "Wideo")
dropdown.pack(pady=5)

tk.Button(root, text="Start", command=lambda: start_analysis(source_var.get())).pack(pady=10)
root.mainloop()

Analiza zakończona. Wyniki zapisane do results.csv
Analiza zakończona. Wyniki zapisane do results.csv
Nie wybrano pliku.


In [5]:
!git add .
